In [ ]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
import numpy as np
import tensorflow as tf
import edward as ed
from edward.models import Normal, Empirical
from keras import datasets
output_notebook()

## DATA

In [ ]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

X_train = X_train.reshape(-1, 28**2)
X_test = X_test.reshape(-1, 28**2)

X_train = X_train/128
X_test = X_test/128

y_train_oh = np.eye(10)[y_train]
y_test_oh = np.eye(10)[y_test]

X_train = X_train.astype('float32')
y_train_oh = y_train_oh.astype('float32')
X_test = X_test.astype('float32')
y_test_oh = y_test_oh.astype('float32')

N, DIN = X_train.shape
DOUT = 10

## MODEL

In [ ]:
def neural_network(X):
    h = tf.tanh(tf.matmul(X, W_0) + b_0)
    h = tf.tanh(tf.matmul(h, W_1) + b_1)
    h = tf.nn.softmax(tf.matmul(h, W_2) + b_2)
    return h


W_0 = Normal(mu=tf.zeros([DIN, 10]), sigma=tf.ones([DIN, 10]))
W_1 = Normal(mu=tf.zeros([10, 10]), sigma=tf.ones([10, 10]))
W_2 = Normal(mu=tf.zeros([10, DOUT]), sigma=tf.ones([10, DOUT]))
b_0 = Normal(mu=tf.zeros(10), sigma=tf.ones(10))
b_1 = Normal(mu=tf.zeros(10), sigma=tf.ones(10))
b_2 = Normal(mu=tf.zeros(DOUT), sigma=tf.ones(DOUT))

X = tf.cast(X_train, dtype=tf.float32)
y = Normal(mu=neural_network(X), sigma=0.1 * tf.ones([N, DOUT]))

## INFERENCE

In [ ]:
N_SAMPLES = 100
qW_0 = Empirical(params=tf.Variable(tf.random_normal([N_SAMPLES, DIN, 10])))
qW_1 = Empirical(params=tf.Variable(tf.random_normal([N_SAMPLES, 10, 10])))
qW_2 = Empirical(params=tf.Variable(tf.random_normal([N_SAMPLES, 10, DOUT])))
qb_0 = Empirical(params=tf.Variable(tf.random_normal([N_SAMPLES, 10])))
qb_1 = Empirical(params=tf.Variable(tf.random_normal([N_SAMPLES, 10])))
qb_2 = Empirical(params=tf.Variable(tf.random_normal([N_SAMPLES, DOUT])))

In [ ]:
inference = ed.HMC({W_0: qW_0, b_0: qb_0,
                    W_1: qW_1, b_1: qb_1,
                    W_2: qW_2, b_2: qb_2}, data={X: X_train, y: y_train_oh})

In [ ]:
inference.run()

## CRITICISM

In [ ]:
h = tf.tanh(tf.matmul(X_test, qW_0) + qb_0)
h = tf.tanh(tf.matmul(h, qW_1) + qb_1)
h = tf.nn.softmax(tf.matmul(h, qW_2) + qb_2)

In [ ]:
y_pred = h.eval()

In [ ]:
(y_test == y_pred.argmax(axis=1)).mean()